# Background


Loop through the entire segmented area and generate all the models.

I found the database here:
 - [NeuroMouse Project](http://phm.utoronto.ca/~jeffh/surgical.htm)
 - [Back to Mouse Atlas Variational Mouse Brain Atlas](http://localhost:8888/notebooks/notebooks/Segmentation/Dataset%20-%20Mouse%20Brain%20and%20Labels%20V02.ipynb) This is the other toronto based atlas.  hopefully the labels here are accurate.
 

# Imports

In [1]:
import nibabel as nib  
import numpy
import matplotlib.pyplot as plt
%pylab inline
from IPython.display import Image

Populating the interactive namespace from numpy and matplotlib


In [2]:
import scipy.misc
from skimage import measure

In [3]:
# for this module the stl is now depricated
# pip install numpy-stl
from stl import mesh

In [4]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Functions

In [5]:
def loadMNC(filepath):
    img = nib.load(filepath)
    data = img.get_data()
    return data

In [6]:
def showSlice(data):
    plt.set_cmap("gray")
    plt.imshow(data)

In [7]:
def getLabelInfo(labelArray,index):
    try:
        print(labelArray[index])
    except:
        print("{} is too large the there are only {} labels including 0 index".format(index,len(labelArray)-1))
    

In [8]:
def make_mesh(image, threshold=-300, step_size=1):
#     print "Transposing surface"
    p = image.transpose(2,1,0)
    
#     print "Calculating surface"
    try:
        verts, faces = measure.marching_cubes(p, threshold) 
    except:
        verts, faces = measure.marching_cubes_classic(p, threshold)         
    return verts, faces

# Parameters

In [9]:
# Directory locations
pathBrain = "C://Cerebral//_MarioSandBox//DB//medicalHolography//Animal Brains//UofTLabeledData//129SV_atlas.mnc"
pathLabels = "C://Cerebral//_MarioSandBox//DB//medicalHolography//Animal Brains//UofTLabeledData//129SV_atlas_labelling.mnc"
pathLabelInfo = "C://Cerebral//_MarioSandBox//DB//medicalHolography//Animal Brains//UofTLabeledData//129SV_atlas_labelling_names.txt"

In [10]:
areaInd = 3 # Use the text file "129SV_atlas_labelling_names.txt" to select the approriate area you want to create an STL of 
outDir = 'C://Cerebral//_MarioSandBox//DB//medicalHolography//Animal Brains//UofTLabeledData//Segmented//'

# Reading File

Read the label informaiton and place it in an array for easy viewing later

In [21]:
labelInfo = []

f = open(pathLabelInfo, "r") 
for line in f: 
    labelInfo.append(line)
#     print line 

In [12]:
getLabelInfo(labelInfo,areaInd-1)

3 brain_stem



There are some nodes that do not have labels so keep track of these as well just in case

In [13]:
dataBrain = loadMNC(pathBrain)
dataLabels = loadMNC(pathLabels)

# Segmentation

Crude segmentation when we have the labels:
1. Obtain label slice and data slice
2. create mask of the label
3. multiply mask and data
4. store new masked data
5. repeat for all slices

- dataBrain = variable containing the data
- dataLabels = varible containg the labels

In [14]:
len(labelInfo)

43

In [15]:
labelInfoLine = labelInfo[0]

In [18]:
for areaOfInterest in labelInfo:
    
    print('Now processing {}'.format(areaOfInterest))
    areaInd,locationName = areaOfInterest.split()
    
    areaInd = int(areaInd)
    
    numSlice,xdim,ydim = dataBrain.shape
    
    dataBrainSegmented = np.zeros(dataBrain.shape)

    for sliceInd in xrange(numSlice):
    #     print('Now applying mask to slice {}'.format(sliceInd))
        # looping paramters
        # this will be the cerebellum
        # sliceInd = 80

        # obtain working slice
        currentDataSlice = np.copy(dataBrain[sliceInd,:,:])
        currentLabelSlice = np.copy(dataLabels[sliceInd,:,:])

        # create the masks
        currentLabelSlice[np.where(currentLabelSlice!=areaInd)] = 0
        currentLabelSlice[np.where(currentLabelSlice==areaInd)] = 1

        # apply mask
        maskedData = currentLabelSlice * currentDataSlice

        # store mask in the volumetric data
        dataBrainSegmented[sliceInd,:,:] = maskedData

        # Save images    
        #     scipy.misc.imsave(outDir+'{}.jpg'.format(sliceInd), maskedData)
        # we are not doing this... we want to save them in an stl file instead of a stack of images

    # Generate mesh
    v, f = make_mesh(dataBrainSegmented,0.0)

    # generate volume
    veins = mesh.Mesh(np.zeros(f.shape[0], dtype=mesh.Mesh.dtype))

    for i, f in enumerate(f):
        for j in range(3):
            veins.vectors[i][j] = v[f[j],:]

    # save volume
    outSTLDir = 'C://Cerebral//_MarioSandBox//DB//medicalHolography//Animal Brains//UofTLabeledData//Segmented//'
    veins.save(outSTLDir+locationName+'.stl')
    

Now processing 1 cortex

Now processing 2 cerebellum

Now processing 3 brain_stem

Now processing 4 diencephalon

Now processing 5 olfactory_bulbs

Now processing 6 hippocampus

Now processing 7 striatum

Now processing 8 corpus_callosum

Now processing 9 superior_colliculus

Now processing 10 periaqueductal_gray

Now processing 11 fimbria

Now processing 12 internal_capsule

Now processing 13 dentate_gyrus

Now processing 14 cerebral_penduncle

Now processing 15 anterior_commissure_pars_anterior

Now processing 16 optic_tract

Now processing 17 stria_medullaris_thalami

Now processing 18 medial_lemniscus_longitudinal_fasciculus

Now processing 19 medial_lemniscus

Now processing 20 facial_nerve 

Now processing 21 mammilothalamic_tract

Now processing 22 fornix

Now processing 23 anterior_commissure_pars_posterior

Now processing 25 habenulo_peduncular_tract 

Now processing 26 posterior_commissure

Now processing 43 inferior_cilliculus

Now processing 44 globus_pallidus

Now processi

ValueError: too many values to unpack

There was a problem with the spacing in the text file but this was fixed using the following code.

In [28]:
counter = 0

for areaOfInterest in labelInfo:
    if counter>27:
        print('Now processing {}'.format(areaOfInterest))
        areaInd,locationName = areaOfInterest.split()

        areaInd = int(areaInd)

        numSlice,xdim,ydim = dataBrain.shape

        dataBrainSegmented = np.zeros(dataBrain.shape)

        for sliceInd in xrange(numSlice):
        #     print('Now applying mask to slice {}'.format(sliceInd))
            # looping paramters
            # this will be the cerebellum
            # sliceInd = 80

            # obtain working slice
            currentDataSlice = np.copy(dataBrain[sliceInd,:,:])
            currentLabelSlice = np.copy(dataLabels[sliceInd,:,:])

            # create the masks
            currentLabelSlice[np.where(currentLabelSlice!=areaInd)] = 0
            currentLabelSlice[np.where(currentLabelSlice==areaInd)] = 1

            # apply mask
            maskedData = currentLabelSlice * currentDataSlice

            # store mask in the volumetric data
            dataBrainSegmented[sliceInd,:,:] = maskedData

            # Save images    
            #     scipy.misc.imsave(outDir+'{}.jpg'.format(sliceInd), maskedData)
            # we are not doing this... we want to save them in an stl file instead of a stack of images

        # Generate mesh
        v, f = make_mesh(dataBrainSegmented,0.0)

        # generate volume
        veins = mesh.Mesh(np.zeros(f.shape[0], dtype=mesh.Mesh.dtype))

        for i, f in enumerate(f):
            for j in range(3):
                veins.vectors[i][j] = v[f[j],:]

        # save volume
        outSTLDir = 'C://Cerebral//_MarioSandBox//DB//medicalHolography//Animal Brains//UofTLabeledData//Segmented//'
        veins.save(outSTLDir+locationName+'.stl')
    
    counter = counter +1

Now processing 46 3rd_and_4th_ventricle_cerebral_acquaduct 

Now processing 47 arbor_vita

Now processing 48 pre_para_subiculum

Now processing 49 UD1 

Now processing 50 UD2 

Now processing 51 amygdaloid_region_ventral 

Now processing 52 ventral_pallidm_&_medial_forebtain_bundle 

Now processing 53 medial_septum

Now processing 54 lateral_septum

Now processing 55 nucleus_accumbens_undefined_area 

Now processing 56 UD3

Now processing 57 UD4 

Now processing 58 amygdaloid_region_dorsal 

Now processing 59 stria_terminallis

Now processing 60 UD5

